In [25]:
# estimation of Adam learning model parameters under rational expectations

In [26]:
# E(r^s) = a
# Var(r^s) = a ** 2 * exp(s_div ** 2 - 1)
# E(PD) = delta * a ** (1 - gamma) * rho / (1 - delta * a ** (1 -gamma) * rho)
# E(r^f) = delta ** -1 * a ** -gamma
# delta = a ** 

In [27]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.io import loadmat

In [28]:
data = loadmat("SeriesDATA.mat")

In [29]:
def rho_e(gamma, sc, sd, rho_sc_sd):
    return np.exp(gamma * (1 + gamma) * sc**2 / 2) * np.exp(-gamma * rho_sc_sd)

def price_dividend_ratio(gamma, a, sd, sc, rho_sc_sd, delta):
    rhoe = rho_e(gamma, sc, sd, rho_sc_sd)
    numerator = delta * a**(1 - gamma) * rhoe
    denominator = 1 - a**(1 - gamma) * rhoe
    return numerator / denominator

def discount_factor(a, gamma, sc, rf):
    return rf**(-1) * (a**gamma * np.exp(-gamma * (1 + gamma) * sc**2 / 2))

In [30]:
def calibrate_model(avg_gross_return, var_gross_return, 
                    price_dividend_data, sc_ratio=1/7, rho=0.2, rf=1.01):
    """
    Calibrates [a, s_d, δ, γ] to match empirical moments.
    
    Inputs:
    - avg_gross_return : mean(P_t / P_{t-1}) from data
    - var_gross_return : var(P_t / P_{t-1}) from data
    - price_dividend_data : empirical P/D ratio from data
    - sc_ratio : s_c / s_d (default 1/7)
    - rho : correlation between shocks
    - rf : observed gross risk-free rate
    """
    
    # Step 1: a = average gross return
    a = avg_gross_return
    
    # Step 2: s_d from variance equation: Var(P_t/P_{t−1}) = a^2 (e^{s_d^2} − 1)
    sd = np.sqrt(np.log(1 + var_gross_return / a**2))
    
    # Step 3: s_c = s_d / 7 and ρ = 0.2 (given)
    sc = sd * sc_ratio
    rho_sc_sd = rho * sc * sd
    
    # Step 4 & 5: Solve for γ such that model P/D ratio = empirical P/D ratio
    def equilibrium_condition(gamma):
        delta = discount_factor(a, gamma, sc, rf)
        model_pd = price_dividend_ratio(gamma, a, sd, sc, rho_sc_sd, delta)
        return model_pd - price_dividend_data

    sol = root_scalar(equilibrium_condition, bracket=[0.1, 1000], method='brentq')

    if not sol.converged:
        raise RuntimeError("Calibration failed: no solution for gamma.")

    gamma_star = sol.root
    delta_star = discount_factor(a, gamma_star, sc, rf)
    
    

    return {
        "a": a,
        "s_d": sd,
        "s_c": sc,
        "rho": rho,
        "delta": delta_star,
        "gamma": gamma_star
    }

In [32]:
if __name__ == "__main__":
    # Example calibration with arbitrary data
    empirical_avg_return = np.mean(data['rs'])      # average gross return
    empirical_var_return = np.var(data['rs'])       # variance of gross return
    empirical_price_dividend = np.mean(data['PD'])  # price-dividend ratio
    risk_free_rate = float(np.mean(1 + data["rb"]))  # gross monthly rf
        # 2% risk-free rate

    params = calibrate_model(empirical_avg_return, empirical_var_return,
                             empirical_price_dividend, rf=risk_free_rate)
    
    print("=== Calibrated Parameters ===")
    for k, v in params.items():
        print(f"{k:>8s}: {v:.4f}")

ValueError: f(a) and f(b) must have different signs

In [ ]:
print("Mean return:", np.mean(data['rs']))
print("Var return:", np.var(data['rs']))
print("Mean P/D:", np.mean(data['PD'])/12)
print("Mean risk-free rate:", np.mean(data['rb']))

In [33]:
def _diagnose_equilibrium(eq_fn):
    xs = np.logspace(-3, 3, 60)  # gamma from 0.001 to 1000
    vals = []
    for x in xs:
        try:
            v = eq_fn(x)
            if not np.isfinite(v):
                v = np.nan
        except Exception:
            v = np.nan
        vals.append(v)
    finite = np.array([np.isfinite(v) for v in vals])
    print(f"Finite evaluations: {finite.sum()}/{len(xs)}")
    if finite.sum():
        vmin, vmax = np.nanmin(vals), np.nanmax(vals)
        print(f"Range of f(gamma): [{vmin:.4g}, {vmax:.4g}]")
    # report sign changes
    sign = np.sign(vals)
    for i in range(len(xs)-1):
        if np.isfinite(vals[i]) and np.isfinite(vals[i+1]) and sign[i] != 0 and sign[i+1] != 0 and sign[i] != sign[i+1]:
            print(f"Sign change near gamma in [{xs[i]:.4g}, {xs[i+1]:.4g}]")
            break
    return xs, vals

# Run it
xs, vals = _diagnose_equilibrium(equilibrium_condition)
print("f(0.1) =", equilibrium_condition(0.1))
print("f(1000) =", equilibrium_condition(1000.0))

NameError: name 'equilibrium_condition' is not defined

In [ ]:
import matplotlib.pyplot as plt
a = 1.0216019489023302
sd = np.sqrt(np.log(1 + 0.01292777128836891 / a**2))
sc = sd/7
rho_sc_sd = 0.2 * sc * sd
rf = 1.001331009653297
pd_data = 127.67504604787156

gammas = np.linspace(0.01, 20, 400)
vals = [price_dividend_ratio(g, a, sd, sc, rho_sc_sd, discount_factor(a, g, sc, rf)) for g in gammas]

plt.plot(gammas, vals)
plt.axhline(pd_data, color='r', linestyle='--')
plt.xlabel('γ')
plt.ylabel('Model P/D')
plt.title('Model vs Data P/D')
plt.grid(True)
plt.show()